# Database test

In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
DB_NAME = "jobFinder"
COLLECTION_NAME = "cv_mehdi"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_search_index"
MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGO_URI")

In [3]:
mongo_client = MongoClient(MONGODB_ATLAS_CLUSTER_URI, server_api=ServerApi('1'))
collection = mongo_client[DB_NAME][COLLECTION_NAME]
print(collection)

Collection(Database(MongoClient(host=['ac-wzyenof-shard-00-01.byabgoe.mongodb.net:27017', 'ac-wzyenof-shard-00-02.byabgoe.mongodb.net:27017', 'ac-wzyenof-shard-00-00.byabgoe.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-u5o2v0-shard-0', tls=True, server_api=<pymongo.server_api.ServerApi object at 0x0000022EBEA24160>), 'jobFinder'), 'cv_mehdi')


In [4]:
embeddings = OpenAIEmbeddings()#disallowed_special=())
vector_search = MongoDBAtlasVectorSearch(collection, embedding=embeddings, index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME, embedding_key="embedding")

In [5]:
try:
    mongo_client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


vectera

In [6]:
query = "certificats"

vector_search.similarity_search_with_score(query, k=2)

[(Document(page_content='\nCertificats :\nDeepLearning.AI : Generative AI with Large Language Models [Lien : https://www.coursera.org/account/accomplishments/certificate/NSH3AYC4NUWR]\nUdacity : Data Analysis Track [Lien : https://drive.google.com/file/d/1NgOzwTS8l3QH5Xu1WLxzX50RJRjxnXgZ/view]\n', metadata={'_id': ObjectId('658205b96a0b4504abfce48c'), 'embedding': [-0.014884235501229302, -0.013048826044008881, 0.030491918615826768, -0.018662231943353112, -0.00797800399204572, 0.02173017970929887, -0.03137612926109055, 0.008768436046816638, 0.0038282361701596655, -0.018394290452452, 0.030920626863913604, 0.0222660664163912, -0.016210553331770027, -0.00334091467320037, 0.0005404913402867882, 0.0055497700280858495, 0.022600996073985174, -0.019881374108855918, 0.015741652928725498, -0.006661733734583156, -0.011930163334577783, 0.01611677287863211, -0.026941672440968904, -0.026780907546428236, 0.007643075265774276, 0.007676567952136915, 0.01227179106378302, -0.03376082156862583, 0.003881824

In [7]:
prompt_template ="""
    You are a smart agent that can answer questions about CVs.
    if the question is not related to CVs, you should answer "I don't know please refer to Mehdi for more information".
    if the question is related to CVs, you should answer the question.
    If the question was not clear enough, you should ask for more information.
    if the question was in French, you should answer in French.
    use the following information to answer the question:
    {context}
    question: {question}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [8]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa_retriever = vector_search.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
    return_vector=False,
)
llm = OpenAI(temperature=0.3, max_tokens=200, top_p=0.9)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=qa_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

docs = qa({"query": "gpt-4 compute requirements"})

print(docs["result"])
print(docs["source_documents"])


I'm sorry, I don't know. Please refer to Mehdi for more information.
[Document(page_content='\nCompétences :\nLangages de programmation Python : 6ans C++, C : 6ans Matlab: 4 ans JavaScript : 3 ans\nFramework Front: React, Svelte Back : Flask,streamlit, FastAPI ML : tensorflow, Pytorch Version control: git , Pack office (power point word excel)\n', metadata={'_id': ObjectId('658205b96a0b4504abfce489'), 'embedding': [-0.00300439002896078, -0.009004726017982072, 0.007829315911148526, -0.01919837283930263, -0.014267052434029556, 0.0061641502737607, -0.016590854048654465, 0.010058542650238833, -0.010227423096921632, -0.006944379725574564, 0.033451914576827245, 0.017901368326569734, -0.0038065740133492133, -0.008829089757385515, -0.0009533322821927607, 0.004583425791036841, 0.023521723077724085, -0.008106280067225734, -0.010470611908720335, -0.00317495966195266, 0.00643435954724672, -0.004127447467406202, 0.0028270650477164277, -0.007126771334423591, -0.01737445954478007, 0.0070524634349689

In [9]:
docs = qa({"query": "how old is Mehdi?"})
print(docs["result"])


Mehdi is 23 years old.


# Agent section

In [10]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.agents.react.base import DocstoreExplorer
from langchain.docstore import Wikipedia
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMMathChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from pydantic import BaseModel, Field

In [64]:
llm = ChatOpenAI(temperature=0.2)

In [65]:
qa_retriever = vector_search.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
    return_vector=False,
    return_metadata=False,
)

In [66]:
from langchain.tools import tool
from datetime import datetime

@tool("current_date", return_direct=False)
def current_date(input:str) -> str:
  """Used to know the current date"""
  return datetime.now()

@tool("mehdi_cv_data", return_direct=False)
def mehdi_cv_data(input:str) -> str:
  """Used as bank of data for Mehdi CV the input should be a question related to the task to perform"""
  context_helper = "\n".join([ x[0].page_content for x in vector_search.similarity_search_with_score(input, k=4)])
  return context_helper

@tool("meaning_search", return_direct=False)
def meaning_search(input:str) -> str:
  """Used when you look for the meaning of a word"""
  llm = OpenAI(temperature=0.3, max_tokens=50, top_p=0.9)
  return llm(f"What is {input}?")

@tool("no_answer_found", return_direct=True)
def no_answer_found(input:str) -> str:
  """Used when the question provided by the user was not found in the data bank example : when did Mehdi started working at Google? => no answer found"""
  llm = OpenAI(temperature=0.8, max_tokens=50, top_p=0.9)
  return str(llm(f"answer with something that has the following meaning: I don't know or the input provided does not match any of my database information please refer to Mehdi for more information"))
  

@tool("out_of_context", return_direct=True)
def out_of_context(input:str) -> str:
  """Used when the question provided by the user is not related to the task to perform example : what is the weather today?"""
  message = f"I don't know please refer to Mehdi for more information"
  
  return message


In [67]:
llm = ChatOpenAI(temperature=0.2, model="gpt-4")
tools = load_tools(["llm-math"], llm=llm)
tools.append(current_date)
tools.append(mehdi_cv_data)
tools.append(meaning_search)
tools.append(no_answer_found)
tools.append(out_of_context)
# tools.append(answer_found)

In [68]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [69]:
agent.run("Quand est-ce que Mehdi à commancer à travailler chez BCG?")



> Entering new AgentExecutor chain...
Je dois trouver des informations sur l'expérience professionnelle de Mehdi. Je vais utiliser l'outil mehdi_cv_data pour obtenir ces informations.
Action: mehdi_cv_data
Action Input: Quand est-ce que Mehdi à commancer à travailler chez BCG?
Observation: Bonjour, je suis Mehdi HAYANI MECHKOURI, né le premier janvier 2000. De nationalité marocaine. Actuellement à Paris en France. Je suis actuellement en double diplôme arts et métiers ParisTech avec comme expertise, L’ingénierie numérique et Data science et intelligence artificielle. Je suis à la recherche d’un stage de 6 mois à partir de février 2023 dans le domaine de l’IA et la data science. Je suis une personne curieuse avec une forte soif à la connaissance. Je suis une personne qui cherche à résoudre les problèmes spécialement les problèmes liés au changement climatique, la sécurité alimentaire, et la sécurité énergétique. Mais je suis prêt à affronter n’importe quel défi. Mon secteur de prédile

"\n\nI'm sorry, I'm not familiar with that. For more information, please contact Mehdi."

In [70]:
agent.run("Quel certificats Mehdi a obtenu?")



> Entering new AgentExecutor chain...
The question is asking about the certificates that Mehdi has obtained. This information should be in Mehdi's CV data.
Action: mehdi_cv_data
Action Input: Quel certificats Mehdi a obtenu?
Observation: 
Certificats :
DeepLearning.AI : Generative AI with Large Language Models [Lien : https://www.coursera.org/account/accomplishments/certificate/NSH3AYC4NUWR]
Udacity : Data Analysis Track [Lien : https://drive.google.com/file/d/1NgOzwTS8l3QH5Xu1WLxzX50RJRjxnXgZ/view]

Bonjour, je suis Mehdi HAYANI MECHKOURI, né le premier janvier 2000. De nationalité marocaine. Actuellement à Paris en France. Je suis actuellement en double diplôme arts et métiers ParisTech avec comme expertise, L’ingénierie numérique et Data science et intelligence artificielle. Je suis à la recherche d’un stage de 6 mois à partir de février 2023 dans le domaine de l’IA et la data science. Je suis une personne curieuse avec une forte soif à la connaissance. Je suis une personne qui ch

'Mehdi a obtenu les certificats suivants : \n1. DeepLearning.AI : Generative AI with Large Language Models. Vous pouvez voir le certificat ici : https://www.coursera.org/account/accomplishments/certificate/NSH3AYC4NUWR\n2. Udacity : Data Analysis Track. Vous pouvez voir le certificat ici : https://drive.google.com/file/d/1NgOzwTS8l3QH5Xu1WLxzX50RJRjxnXgZ/view'

In [71]:
agent.run("Quel est le nom de l'entreprise où Mehdi a travaillé en tant que consultant?")



> Entering new AgentExecutor chain...
Je dois trouver des informations sur l'expérience professionnelle de Mehdi. Je vais utiliser la fonction mehdi_cv_data pour obtenir ces informations.
Action: mehdi_cv_data
Action Input: Où Mehdi a-t-il travaillé en tant que consultant?
Observation: Bonjour, je suis Mehdi HAYANI MECHKOURI, né le premier janvier 2000. De nationalité marocaine. Actuellement à Paris en France. Je suis actuellement en double diplôme arts et métiers ParisTech avec comme expertise, L’ingénierie numérique et Data science et intelligence artificielle. Je suis à la recherche d’un stage de 6 mois à partir de février 2023 dans le domaine de l’IA et la data science. Je suis une personne curieuse avec une forte soif à la connaissance. Je suis une personne qui cherche à résoudre les problèmes spécialement les problèmes liés au changement climatique, la sécurité alimentaire, et la sécurité énergétique. Mais je suis prêt à affronter n’importe quel défi. Mon secteur de prédilectio

"\n\nI'm sorry, I don't have the answer to that. Please contact Mehdi for more information."

In [72]:
agent.run("Quel était le poste de Mehdi chez GE grid solutions?")



> Entering new AgentExecutor chain...
The question is asking for information from Mehdi's CV.
Action: mehdi_cv_data
Action Input: Quel était le poste de Mehdi chez GE grid solutions?
Observation: Bonjour, je suis Mehdi HAYANI MECHKOURI, né le premier janvier 2000. De nationalité marocaine. Actuellement à Paris en France. Je suis actuellement en double diplôme arts et métiers ParisTech avec comme expertise, L’ingénierie numérique et Data science et intelligence artificielle. Je suis à la recherche d’un stage de 6 mois à partir de février 2023 dans le domaine de l’IA et la data science. Je suis une personne curieuse avec une forte soif à la connaissance. Je suis une personne qui cherche à résoudre les problèmes spécialement les problèmes liés au changement climatique, la sécurité alimentaire, et la sécurité énergétique. Mais je suis prêt à affronter n’importe quel défi. Mon secteur de prédilection est la programmation et je suis fasciné par le potentiel de l’IA.


Contacts :
Téléphone 

'Mehdi était un Full Stack Developer chez GE Grid Solutions.'

In [73]:
agent.run("Dans quel ville réside Mehdi? et quand est-ce qu'il a commancé ça formation dans cette ville?")



> Entering new AgentExecutor chain...
The question is asking for the city where Mehdi lives and when he started his training in that city. This information can be found in Mehdi's CV.
Action: mehdi_cv_data
Action Input: "Where does Mehdi live?" and "When did Mehdi start his training in his city of residence?"
Observation: Bonjour, je suis Mehdi HAYANI MECHKOURI, né le premier janvier 2000. De nationalité marocaine. Actuellement à Paris en France. Je suis actuellement en double diplôme arts et métiers ParisTech avec comme expertise, L’ingénierie numérique et Data science et intelligence artificielle. Je suis à la recherche d’un stage de 6 mois à partir de février 2023 dans le domaine de l’IA et la data science. Je suis une personne curieuse avec une forte soif à la connaissance. Je suis une personne qui cherche à résoudre les problèmes spécialement les problèmes liés au changement climatique, la sécurité alimentaire, et la sécurité énergétique. Mais je suis prêt à affronter n’importe 

"\n\nI'm sorry, I'm not familiar with that. Please ask Mehdi for more information."

In [74]:
agent.run("Combien de langues parle Mehdi ?")




> Entering new AgentExecutor chain...
Je dois chercher cette information dans le CV de Mehdi.
Action: mehdi_cv_data
Action Input: Combien de langues parle Mehdi ?
Observation: Bonjour, je suis Mehdi HAYANI MECHKOURI, né le premier janvier 2000. De nationalité marocaine. Actuellement à Paris en France. Je suis actuellement en double diplôme arts et métiers ParisTech avec comme expertise, L’ingénierie numérique et Data science et intelligence artificielle. Je suis à la recherche d’un stage de 6 mois à partir de février 2023 dans le domaine de l’IA et la data science. Je suis une personne curieuse avec une forte soif à la connaissance. Je suis une personne qui cherche à résoudre les problèmes spécialement les problèmes liés au changement climatique, la sécurité alimentaire, et la sécurité énergétique. Mais je suis prêt à affronter n’importe quel défi. Mon secteur de prédilection est la programmation et je suis fasciné par le potentiel de l’IA.


Contacts :
Téléphone +33 7 52 07 57 72, e

"Mehdi parle quatre langues : le français, l'anglais, l'arabe et l'allemand."

In [75]:
agent.run("es-ce que Mehdi a une experience en data science?")



> Entering new AgentExecutor chain...
The question is asking about Mehdi's experience in data science. I should check Mehdi's CV data.
Action: mehdi_cv_data
Action Input: Does Mehdi have experience in data science?
Observation: Bonjour, je suis Mehdi HAYANI MECHKOURI, né le premier janvier 2000. De nationalité marocaine. Actuellement à Paris en France. Je suis actuellement en double diplôme arts et métiers ParisTech avec comme expertise, L’ingénierie numérique et Data science et intelligence artificielle. Je suis à la recherche d’un stage de 6 mois à partir de février 2023 dans le domaine de l’IA et la data science. Je suis une personne curieuse avec une forte soif à la connaissance. Je suis une personne qui cherche à résoudre les problèmes spécialement les problèmes liés au changement climatique, la sécurité alimentaire, et la sécurité énergétique. Mais je suis prêt à affronter n’importe quel défi. Mon secteur de prédilection est la programmation et je suis fasciné par le potentiel 

"Oui, Mehdi a une expérience en data science. Il poursuit actuellement un double diplôme avec une expertise en ingénierie numérique et en science des données et intelligence artificielle. Il est également à la recherche d'un stage de 6 mois dans le domaine de l'IA et de la science des données. Il a également suivi une formation en analyse de données chez Udacity."

## Conclusion

The agent gives good answers overall but a change in the data and adding meta deta can improve the retrived data